# <center>Phân tích ma trận trong Python</center>

## Mục lục
* [Thực hành](#c1)
    * [Trị riêng - Vector riêng](#c11)
    * [Chéo hóa](#c12)
    * [Chéo hóa trực giao](#c13)
* [Đồ án 2: Image Processing](#c2)
    * [Nội dung đồ án](#c21)
    * [Quy định nộp bài](#c22)
    * [Quy định chấm bài](#c23)

## Thực hành <a class="anchor" id="c1"></a>

Trong lab này, chúng ta sẽ tìm hiểu về phân tích ma trận sử dụng `NumPy`.

Cho ma trận:
$$A = \begin{bmatrix}
    3 & -2 & 0\\ 
    -2 & 3 & 0\\ 
    0 & 0 & 5
    \end{bmatrix}$$

In [1]:
import numpy as np

In [2]:
A = np.array([[3, -2, 0],
              [-2, 3, 0],
              [0, 0, 5]])

Các phân tích được giới thiệu trong lab này là:
1. Tìm trị riêng và vector riêng
2. Chéo hóa
3. Chéo hóa trực giao

### Trị riêng - Vector riêng <a class="anchor" id="c11"></a>

#### Thuật toán [Power iteration](https://en.wikipedia.org/wiki/Power_iteration)

In [3]:
def eigen_power_iteration(A, n_iter=1000, eps=10e-5):
    # Khởi tạo vector ngẫu nhiên
    b_k = np.random.rand(A.shape[1])
    b_k_pre = np.zeros(A.shape[1])

    # Tìm vector riêng
    for _ in range(int(n_iter)):
        # Tính tích ma trận theo vector riêng
        numerator = np.dot(A, b_k)
        denominator = np.linalg.norm(numerator)
        
        b_k = numerator / denominator
        
        # Dừng khi sự thay đổi của vecto trước và vector sau không đáng kể
        if np.all(np.abs(b_k - b_k_pre) < eps):
            break
        
        b_k_pre = b_k
        
    # Tìm trị riêng (Rayleigh quotient iteration)
    lamb = ((b_k @ A) @ b_k) / (b_k @ b_k)

    return lamb, b_k


def my_eigens(A, n_iter=1000, eps=10e-5):
    eigenvalues = []
    eigenvectors = []
    
    n_rows = A.shape[0]
    
    for _ in range(n_rows):
        lamb, b_k = eigen_power_iteration(A, n_iter, eps)
        
        eigenvalues.append(lamb)
        eigenvectors.append(b_k)
        
        # Hotelling’s deflation
        b_k = b_k.reshape(n_rows, -1)
        A = A - (lamb/np.square(np.linalg.norm(b_k))) * (b_k @ b_k.T)
        
    return np.array(eigenvalues), np.array(eigenvectors).T

In [4]:
my_eigens(A)

(array([5.        , 5.        , 1.00000001]),
 array([[ 2.66733097e-01, -6.54868475e-01,  7.07109379e-01],
        [-2.66705210e-01,  6.54880957e-01,  7.07104175e-01],
        [ 9.26132704e-01,  3.77197844e-01, -1.07959958e-04]]))

#### Thư viện `np.linalg`

In [5]:
np.linalg.eig(A)

(array([5., 1., 5.]),
 array([[ 0.70710678,  0.70710678,  0.        ],
        [-0.70710678,  0.70710678,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]))

### Chéo hóa <a class="anchor" id="c12"></a>

In [6]:
def my_diag(A):
    # Tìm trị riêng, vector riêng
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    # Sắp xếp giảm dần theo trị riêng
    sorted_idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_idx]
    eigenvectors = eigenvectors[:, sorted_idx]
    
    P_matrix = eigenvectors
    P_inv_matrix = np.linalg.inv(P_matrix)
    D_matrix = np.diag(eigenvalues)
    
    return P_matrix, D_matrix, P_inv_matrix

In [7]:
def is_close(A, B, eps=10e-9):
    return np.all(np.abs(A - B) < eps)

In [8]:
P, D, P_inv = my_diag(A)
(P, D, P_inv)

(array([[ 0.        ,  0.70710678,  0.70710678],
        [ 0.        , -0.70710678,  0.70710678],
        [ 1.        ,  0.        ,  0.        ]]),
 array([[5., 0., 0.],
        [0., 5., 0.],
        [0., 0., 1.]]),
 array([[ 0.        ,  0.        ,  1.        ],
        [ 0.70710678, -0.70710678,  0.        ],
        [ 0.70710678,  0.70710678,  0.        ]]))

In [9]:
is_close(D, P_inv @ A @ P)

True

### Chéo hóa trực giao <a class="anchor" id="c13"></a>

In [10]:
def my_orth_diag(A):
    # Tìm trị riêng, vector riêng
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    # Sắp xếp theo trị riêng giảm dần
    sorted_idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_idx]
    eigenvectors = eigenvectors[:, sorted_idx]
    
    # Trực giao
    P_matrix, _ = np.linalg.qr(eigenvectors)
    
    P_transpose = P_matrix.T
    D_matrix = np.diag(eigenvalues)
    
    return P_matrix, D_matrix, P_transpose

In [11]:
(P, D, P_T) = my_orth_diag(A)
(P, D, P_T)

(array([[ 0.        ,  0.70710678, -0.70710678],
        [-0.        , -0.70710678, -0.70710678],
        [-1.        ,  0.        ,  0.        ]]),
 array([[5., 0., 0.],
        [0., 5., 0.],
        [0., 0., 1.]]),
 array([[ 0.        , -0.        , -1.        ],
        [ 0.70710678, -0.70710678,  0.        ],
        [-0.70710678, -0.70710678,  0.        ]]))

In [12]:
is_close(D, P_T @ A @ P)

True

---

## Đồ án 2: Image Processing <a class="anchor" id="c2"></a>

### Nội dung đồ án <a class="anchor" id="c21"></a>

Nhắc lại: Trong đồ án 1, bạn đã được giới thiệu rằng ảnh được lưu trữ dưới dạng ma trận các điểm ảnh. Mỗi điểm ảnh có thể là một giá trị (ảnh xám) hoặc một vector (ảnh màu).

Trong đồ án này, bạn được yêu cầu thực hiện các chức năng xử lý ảnh cơ bản sau:
    
    1. Thay đổi độ sáng cho ảnh (1 điểm)

![img](https://scontent-hkg4-1.xx.fbcdn.net/v/t1.0-9/109581859_2795629400670258_8018526615084170772_o.jpg?_nc_cat=111&_nc_sid=730e14&_nc_ohc=YljndAVFjOEAX_Otr2o&_nc_oc=AQndVCKq_5wiDxKwI-pvFBP1fP7vU-woCMjlVqQ7X4SLL75ngMu8ZxjuRShCB62k-tg&_nc_ht=scontent-hkg4-1.xx&oh=afe3a3f31c596c2f77f0b2ce6386e6c3&oe=5F3D6E7A)

    2. Thay đổi độ tương phản (1 điểm)

![img](https://scontent-hkg4-1.xx.fbcdn.net/v/t1.0-9/109905028_2795629407336924_8002404333267704951_o.jpg?_nc_cat=104&_nc_sid=730e14&_nc_ohc=SSbeWLNQFcoAX__xGI_&_nc_ht=scontent-hkg4-1.xx&oh=0cc80b4b7cf3ff4d95ab9fbbb06a1d6d&oe=5F3D5B3B)

    3. Chuyển đổi ảnh RGB thành ảnh xám (2 điểm)

![img](https://scontent-hkg4-1.xx.fbcdn.net/v/t1.0-9/109498666_2795629330670265_2391573143560079748_o.jpg?_nc_cat=102&_nc_sid=730e14&_nc_ohc=y091dMm8hc8AX9Eu-rV&_nc_ht=scontent-hkg4-1.xx&oh=0ff087b3a07c502f52f96c205669c958&oe=5F406E73)

Tham khảo tại [đây](https://www.tutorialspoint.com/dip/grayscale_to_rgb_conversion.htm)

    4. Lật ảnh (ngang - dọc) (2 điểm)

![img](https://scontent-hkg4-1.xx.fbcdn.net/v/t1.0-9/112813285_2795629397336925_473147713696095361_o.jpg?_nc_cat=110&_nc_sid=730e14&_nc_ohc=Ue3Oef45RAEAX-upxns&_nc_ht=scontent-hkg4-1.xx&oh=329cd7a1db2a8e4c1819df905d0c377f&oe=5F404C02)

    5. Chồng 2 ảnh cùng kích thước (2 điểm): chỉ làm trên ảnh xám

![img](https://scontent-hkg4-1.xx.fbcdn.net/v/t1.0-9/109898178_2795629227336942_4696040469968534857_o.jpg?_nc_cat=106&_nc_sid=730e14&_nc_ohc=nRmmvwJpHwUAX-F8WDg&_nc_ht=scontent-hkg4-1.xx&oh=2133a1fc084f04e623db4ccc7846219c&oe=5F3FB073)

    6. Làm mờ ảnh (2 điểm)

![img](https://scontent-hkg4-1.xx.fbcdn.net/v/t1.0-9/109576110_2795629354003596_4489329086960038108_o.jpg?_nc_cat=102&_nc_sid=730e14&_nc_ohc=7rd17GpfFvgAX_0JtIZ&_nc_ht=scontent-hkg4-1.xx&oh=663de4dcf1705bd5df433b6d01971d43&oe=5F400D25)

Tham khảo tại [đây](https://en.wikipedia.org/wiki/Kernel_(image_processing)), phần Box blur hoặc Gaussian blur 3 $\times$ 3

Trong đồ án này, bạn <font style="color:red">**CHỈ ĐƯỢC PHÉP**</font> sử dụng các thư viện sau: `PIL`, `numpy`, `matplotlib`

### Quy định bài nộp <a class="anchor" id="c22"></a>

* Thực hiện toàn bộ bài làm trên 1 tập tin Jupyter Notebook (.ipynb) hoặc Python (.py)


* Bạn nộp tập tin `MSSV.zip` được nén từ thư mục MSSV chứa các tập tin sau:
    1. Báo cáo toàn bộ bài làm: `MSSV.pdf`
    2. Mã nguồn: `MSSV.ipynb` hoặc `MSSV.py`


* Trong đó, nội dung tập tin báo cáo gồm có:
    - Thông tin cá nhân: họ và tên, MSSV
    - Liệt kê các chức năng đã hoàn thành
    - Ý tưởng thực hiện, mô tả các hàm chức năng
    - Hình ảnh kết quả với từng chức năng
    
    
* Ví dụ minh họa cây thư mục bài nộp sau khi giải nén tập tin `MSSV.zip` như sau:
```
MSSV
├── MSSV.pdf
└── MSSV.ipynb
```

### Quy định chấm bài <a class="anchor" id="c23"></a>

Những trường hợp sau đây sẽ bị 0 điểm toàn bộ đồ án:
* Nộp sai quy định
* Không có báo cáo
* Thực thi mã nguồn báo lỗi

<font style="color:red">**LƯU Ý: SAO CHÉP BÀI LÀM CỦA NHAU SẼ BỊ 0 ĐIỂM TOÀN BỘ PHẦN THỰC HÀNH**</font>